In [ ]:
import numpy as np
from tqdm import tqdm
import random
import gymnasium as gym
# from gym.wrappers import Monitor
import glob
import io
import matplotlib.pyplot as plt
from IPython.display import HTML
from collections import deque

In [ ]:
#Setting up the environment
env = gym.make('Taxi-v3', render_mode='ansi')
state, _ = env.reset()

# The state of the environment
print(state)

print("decoded state")
print(list(env.decode(state)))


#The number of states in the environment
print(env.observation_space.n)

#The number of actions in the environment
print(env.action_space.n)

#Take a step in the environment
next_state, reward, done, _, _ = env.step(1)
print("decoded state")
print(list(env.decode(next_state)))

#Render the environment
env.render()

[![Options](https://www.researchgate.net/publication/292208109/figure/fig2/AS:960497125978114@1606011768506/Taxi-problem-and-an-action-hierarchy.gif)]

In [ ]:
# actions:  ['down', 'up', 'right', 'left', 'pickup', 'dropoff']
actions = [0, 1, 2, 3, 4, 5]


goal_states = [[0, 0], [0, 4], [4, 0], [4, 3]]

level_1_options = [0, 1]
# highest level options: ['get', 'put']

get_put_options = [0, 1, 2, 3, 4]
# get options: ['pickup/dropff', 'getR', 'getG', 'getY', 'getB']


navigate_options = [0, 1, 2, 3]
# navigate options: ['down', 'up', 'right', 'left']
Q_level_one_options = np.zeros((500, 2)) # Q-values for level options -> get/put

Q_level_two_options = np.zeros((2, 500, len(get_put_options))) # Q-values for level options -> get/put, toR, toG, toY, toB

Q_level_three_options = np.zeros((4, 25, len(navigate_options))) # Q-values for level options -> down, up, right, left


# Softmax function
def softmax(Q, state, tau):
    q_values = Q[state]
    q_values = q_values / tau
    max_q = np.max(q_values)
    e = np.exp(q_values - max_q)
    dist = e / np.sum(e)
    action = np.random.choice(len(dist), p=dist)
    return action

def epsilon_greedy(Q, state, epsilon):
    if random.uniform(0, 1) < epsilon:
        action = np.random.randint(0, 4)
    else:
        action = np.argmax(Q[state])
    return action

In [ ]:
def decode_state(env, state):
    return list(env.decode(state))

In [ ]:
### HYPERPARAMETERS ###
ALPHA_Q = 0.2
ALPHA_OPTIONS = 0.3
ALPHA_OPTIONS_1 = 0.4
GAMMA = 0.99
TAU = 0.1
DECAY_CONSTANT = 0.995

In [ ]:
def execute_level_1_selection(env, state, Q_level_one_options, Q_level_two_options, Q_level_three_options, option, policy, TAU):
    """
    Execute the level 1 selection

    Args:
    env: the environment
    state: the current state
    level_1_options: the level 1 options
    Q_level_two_options: Q-values for level 2 options
    Q_level_three_options: Q-values for level 3 options
    policy: the policy to use

    Returns:
    next_state: the next state
    reward: the reward
    done: is the episode done
    Q_level_two_options: updated Q-values for level 2 options
    Q_level_three_options: updated Q-values for level 3 options
    """
    
    if option < 2:
        """
        Get the Q-values for the level 2 options based on the option chosen
        Initialize the reward bar to 0
        Initialize the counter to 0
        Initialize the current state to the current state
        Get the Q-values for the chosen option
        Update the Q-values for the chosen option based on the reward and the Q-values for the next state
        """
        optdone = False
        optact = 0
        counter = 0
        reward_bar = 0
        chosen_q = Q_level_two_options[option]
        steps = 0
        while not optdone:
            optact = policy(chosen_q, state, TAU)
            next_state, reward, done, [Q_level_two_options, Q_level_three_options], pick_drop_flag, steps_level_2 = execute_level_2_options(env, Q_level_two_options, Q_level_three_options, state, option, optact, policy, TAU)
            reward_bar += reward * GAMMA**counter
            steps += steps_level_2
            chosen_q[state][optact] += ALPHA_OPTIONS * (reward + GAMMA * np.max(chosen_q[next_state]) - chosen_q[state][optact])
            if pick_drop_flag:
                # print("Enters here")
                optdone = True
                Q_level_one_options[state][option] += ALPHA_Q * (reward_bar - Q_level_one_options[state][option] + GAMMA ** counter * np.max(Q_level_one_options[next_state]))
            counter += 1
            state = next_state
        Q_level_two_options[option] = chosen_q
        return next_state, reward_bar, done, [Q_level_one_options, Q_level_two_options, Q_level_three_options], steps
    # else:
    #     next_state, reward, done, _, _ = env.step(option-2)
    #     Q_level_one_options[state][option] += ALPHA_Q * (reward - Q_level_one_options[state][option] + GAMMA * np.max(Q_level_one_options[next_state]))
    #     steps = 1
    #     return next_state, reward, done, [Q_level_one_options, Q_level_two_options, Q_level_three_options], steps


def execute_level_2_options(env, Q_level_two_options, Q_level_three_options, state, option_level_1, option_level_2, policy, TAU):
    """
    Execute the level 2 options

    Args:
    env: the environment
    Q_level_two_options: Q-values for level 2 options
    Q_level_three_options: Q-values for level 3 options 
    state: the current state
    option: the selected option
    level_1_option: the selected level 1 option
    policy: the policy to use

    Returns:
    next_state: the next state
    reward: the reward
    done: is the episode done
    Q_level_two_options: updated Q-values for level 2 options
    Q_level_three_options: updated Q-values for level 3 options
    """
    optdone = False
    optact = 0
    if option_level_2 == 0: # pickup/dropoff
        """
        Pickup or dropoff
        If level 1 option is get, and option is 0, then pickup(4)
        If level 1 option is put, and option is 0, then dropoff(5)    
        """
        if option_level_1 == 0:
            chosen_action = 4
        elif option_level_1 == 1:
            chosen_action = 5
        steps = 1

        """
        Execute the action if it is pickup or dropoff
        Update the Q-values for the level 2 options based on the reward (500, 5)
        """
        next_state, reward, done, _, _ = env.step(chosen_action)
        Q_level_two_options[option_level_1][state][option_level_2] +=  ALPHA_OPTIONS * (reward + GAMMA * np.max(Q_level_two_options[option_level_1][next_state]) - Q_level_two_options[option_level_1][state][option_level_2])
        """
        If the next state is the goal state, then the option is done
        Returns the next state, reward, done, Q-values for level 2 options, Q-values for level 3 options
        """
        # print("Pickup or dropoff")
        return next_state, reward, done, [Q_level_two_options, Q_level_three_options], True, steps
    
    else: # getR, getG, getY, getB
        """
        If the option is not pickup/dropoff, then it is getR, getG, getY, getB
        Get the index of the chosen action
        Initialize the reward bar to 0
        Initialize the counter to 0
        Initialize the current state to the current state
        Based on the chosen action, get the Q-values for the chosen action (25, 4)
        """
        reward_bar = 0
        chosen_action_index = get_put_options.index(option_level_2) - 1
        counter = 0
        current_state = state
        chosen_q = Q_level_three_options[chosen_action_index]
        while not optdone:
            """
            While the option is not done, do the following:
            Decode the state
            Get the state value
            Get the action based on the policy and chosen Q-values(25, 4), state value, and tau 
            Execute the action
            Get the next state value
            Update the reward bar
            Update the Q-values for the chosen action based on the reward and the Q-values for the next state
            """
            state_value = 5 * decode_state(env, state)[0] + decode_state(env, state)[1]
            optact = policy(chosen_q, state_value, TAU)
            next_state, reward, done, _, _ = env.step(optact)
            next_state_value = 5 * decode_state(env, next_state)[0] + decode_state(env, next_state)[1]
            reward_bar += reward*GAMMA**counter
            if decode_state(env, next_state)[:2] == goal_states[chosen_action_index]:
                """
                If the next state is the goal state, then the option is done
                Update the Q-values for the level 2 options based on the reward bar and the Q-values for the next state based on the option chosen by level 3 options
                """
                # print("Goal state reached")
                optdone = True
                Q_level_two_options[option_level_1][current_state][option_level_2] += ALPHA_OPTIONS * \
                    (reward_bar - Q_level_two_options[option_level_1][current_state][option_level_2] + \
                     GAMMA ** counter * np.max(Q_level_two_options[option_level_1][next_state]))
            GOAL_REWARD = 30 if optdone else 0
            chosen_q[state_value][optact] += ALPHA_OPTIONS_1 * (reward + GOAL_REWARD + GAMMA * np.max(chosen_q[next_state_value]) - chosen_q[state_value][optact])
            counter += 1
            state = next_state
        Q_level_three_options[chosen_action_index] = chosen_q
        steps = counter
        # print(reward_bar)
        return next_state, reward_bar, done, [Q_level_two_options, Q_level_three_options], False, steps
                
            

In [ ]:
#### SMDP Q-Learning
episodes = 10000


def SMDP_Q_Learning(env, Q_level_one_options, Q_level_two_options, Q_level_three_options, policy, TAU, DECAY_CONSTANT, episodes):
    # Iterate over 1000 episodes
    total_rewards = np.zeros((episodes))
    for i in tqdm(range(episodes)):
        state, _ = env.reset()    
        done = False
        # While episode is not over
        episode_steps = 0
        while not done:
            option = softmax(Q_level_one_options, state, TAU)
            # Choose and execute the level 1 option
            next_state, reward, done, value_array, steps = execute_level_1_selection(env, state, Q_level_one_options, Q_level_two_options, Q_level_three_options, option, policy, TAU)
            Q_values_level_1, Q_values_level_2, Q_values_level_3 = value_array
            state = next_state
            total_rewards[i] += reward
            TAU = max(0.01, TAU * DECAY_CONSTANT)
            episode_steps += steps
            # print(episode_steps)

    return Q_values_level_1, Q_values_level_2, Q_values_level_3, total_rewards


In [ ]:
Q_level_one_options, Q_level_two_options, Q_level_three_options, total_rewards = SMDP_Q_Learning(env, Q_level_one_options, Q_level_two_options, Q_level_three_options, epsilon_greedy, TAU, DECAY_CONSTANT, episodes)

In [ ]:
def plot_rewards(rewards):
    avg_rew = []
    std_dev = []
    plt.figure("Reward Plot")
    scores_window = deque(maxlen=300)
    for reward in rewards:
        scores_window.append(reward)
        avg_rew.append(np.mean(scores_window))
        std_dev.append(np.std(scores_window))
    plt.plot(avg_rew)
    plt.fill_between(range(len(avg_rew)), np.subtract(avg_rew, std_dev), np.add(avg_rew, std_dev), color='b', alpha=0.1)
    plt.xlabel('Episodes')
    plt.ylabel('Average Reward')
    plt.ylim(-500, 200)  # Set the Y-axis range

plot_rewards(total_rewards)

In [ ]:
print(np.mean(total_rewards[-100:]))